### Analyzing MEPS-HC Data with SAS® 9.4 M6 
#### By Pradip K. Muhuri, PhD

## Exercise 2

### Objective
* Estimate the following parameters
    * mean number of purchases of opioids
    (i.e., Narcotic analgesics or Narcotic    analgesic combos) 
    per person with one or more purchases of opioids

    * mean total, out-of-pocket, and third-party payer expenses 
    for purchases of opioids per person with one or more purchases of opioids
    
### Data and Analysis
    * Run PROC SUMMARY for prescribed medicined data aggregation
    * Merge 2017 aggregated Prescribed Medicines File 
      with Full-Year Consolidated File for the same year
        
    * Run PROC FREQ for data checks
    * Run PROC SURVEYMEANS for complex survey estimates


### MEPS Prescribed Medicines File 

* Each record represents one household-reported prescribed medicine
  that was  purchased during a given calendar year.
  RXRECIDX LINKIDX TC1S1_1 RXXP17X RXSF17X
* For this analysis, the variables include
   * DUPERSID - PERSON ID (DUID + PID)
   * DRUGIDX  - DRUG ID (DUPERSID + COUNTER)
   * RXRECIDX - UNIQUE RX/PRESCRIBED MEDICINE IDENTIFIER
   * LINKIDX  - ID FOR LINKAGE TO COND/OTH EVENT FILES
   * TC1S1_1  - MULTUM THERAPEUT SUB-SUB-CLASS FOR TC1S1
   * RXXP17X  - SUM OF PAYMENTS RXSF17X-RXOU17X(IMPUTED)
   * RXSF17X  - AMOUNT PAID, SELF OR FAMILY (IMPUTED)
 
 [Read here for all varaiables in this data file.](https://meps.ahrq.gov/data_stats/download_data/pufs/h197a/h197adoc.pdf)
  
* The data included in the 2017 file were collected during
    * the 2017 portion of Round 3, Rounds 4 and 5 for Panel 21
    * Rounds 1, 2 and the 2017 portion of Round 3 for Panel 22


##### Create a temporary analytic precribed medicine data file
* Keep only selected variables
* Subset observations for HERAPEUTIC CLASSIFICATION (TC) CODES FOR Narcotic analgesics or Narcotic analgesic combos 

In [ ]:
OPTIONS nodate nonumber;
LIBNAME CDATA 'C:\DATA';
DATA WORK.DRUG;
  SET CDATA.H197A (KEEP=DUPERSID RXRECIDX LINKIDX TC1S1_1 RXXP17X RXSF17X
                   WHERE=(TC1S1_1 IN (60, 191))); 
RUN;

In [ ]:
options nocenter nodate nonumber;
TITLE "A SAMPLE DUMP FOR PMED RECORDS WITH Narcotic analgesics or Narcotic analgesic combos, 2017";
PROC PRINT DATA=WORK.DRUG (OBS=30);
VAR RXRECIDX LINKIDX TC1S1_1 RXXP17X RXSF17X;
 BY DUPERSID;
RUN;


##### Aggregate the prescribed medicine record-level data to the person-level to get SUM (total) of: 
  * RXXP17X values - SUM OF PAYMENTS RXSF17X-RXOU17X(IMPUTED)
  * RXSF17X values - AMOUNT PAID, SELF OR FAMILY (IMPUTED)
##### Show the listing of person-level observations
  

In [ ]:
options nocenter nodate nonumber;
PROC SUMMARY DATA=WORK.DRUG NWAY;
  CLASS DUPERSID;
  VAR RXXP17X RXSF17X;
  OUTPUT OUT=WORK.PERDRUG (DROP=_TYPE_) sum=TOT OOP;
RUN;

TITLE "A SAMPLE DUMP FOR PERSON-LEVEL EXPENDITURES FOR Narcotic analgesics or Narcotic analgesic combos";
PROC PRINT DATA=PERDRUG (OBS=30);
RUN;

* Create a SAS data set from the person-level prescribed medicine data
      * THIRD_PAYER = TOT - OOP  
      (new variable -> third-party expenses for prescribed medicine)
      
      * Rename _FREQ_ as N_PHRCHASE (# OF PURCHASES PER PERSON)
 
 
* Sort the data set

In [ ]:
options nocenter nodate nonumber;
ods html close;
DATA WORK.PERDRUG2;
 SET PERDRUG  (RENAME=(_FREQ_ = N_PHRCHASE)) ; 
 THIRD_PAYER   = TOT - OOP; 
 RUN;
PROC SORT DATA=WORK.PERDRUG2; BY DUPERSID; RUN;





* Sort the full-year consolidated file by keeping only selected variables

In [ ]:
PROC SORT DATA=CDATA.H201 (KEEP=DUPERSID VARSTR VARPSU PERWT17F) OUT=WORK.H201;
BY DUPERSID; RUN;

#### Merge two person-level files

In [ ]:
DATA  WORK.FY;
MERGE WORK.H201 (IN=AA) 
      WORK.PERDRUG2  (IN=BB KEEP=DUPERSID N_PHRCHASE TOT OOP THIRD_PAYER);
   BY DUPERSID;
   IF AA AND BB THEN SUBPOP = 1; /*PERSONS WITH 1+ Narcotic analgesics or Narcotic analgesic combos */
   ELSE IF AA NE BB THEN DO;   
         SUBPOP         = 2 ;  /*PERSONS WITHOUT ANY PURCHASE OF Narcotic analgesics or Narcotic analgesic combos*/
         N_PHRCHASE  = 0 ;  /*# OF PURCHASES PER PERSON */
         THIRD_PAYER = 0 ;
         TOT         = 0 ;
         OOP         = 0 ;
    END;
    IF AA; 
    LABEL   TOT = 'TOTAL EXPENSES FOR NACROTIC ETC'
            OOP = 'OUT-OF-POCKET EXPENSES'
            THIRD_PAYER = 'TOTAL EXPENSES MINUS OUT-OF-POCKET EXPENSES'
            N_PHRCHASE  = '# OF PURCHASES PER PERSON';
RUN;

##### Delete all temporary SAS data sets from the WORK library.

In [ ]:
title;
proc datasets lib=work nolist kill;
quit; 

Here is an atternative SAS program that deletes temporary SAS data sets specified
in the DELETE statement below.
```
PROC DATASETS LIBRARY=WORK nolist; 
 DELETE DRUG PERDRUG2 H201; 
RUN;
QUIT;
```

#### Code snippet for PROC FORMAT

In [ ]:
options nocenter nodate nonumber nosource notes;
ods html close;
PROC FORMAT;
  VALUE GTZERO
     0         = '0'
     0 <- HIGH = '>0' ;
  VALUE SUBPOP    
     1 = 'PERSONS WITH 1+ Narcotic etc'
     2 = 'OTHERS';
RUN;

#### PROC FREQ for data-checks

In [ ]:
PROC FREQ DATA=WORK.FY;
  TABLES  SUBPOP * N_PHRCHASE * TOT * OOP * THIRD_PAYER / LIST NOPERCENT  MISSING ;
  FORMAT SUBPOP SUBPOP. N_PHRCHASE TOT OOP THIRD_PAYER gtzero. ;
RUN;

#### Run PROC SURVEYMEANS and generate  complex survey estimates for the following:

* mean number of purchases of opioids
    (i.e., Narcotic analgesics or Narcotic analgesic combos) 
    per person with one or more purchases of opioids

* mean total, out-of-pocket, and third-party payer expenses 
    for purchases of opioids per person with one or more purchases of opioids
    


In [ ]:
options nocenter;
ods graphics off; /*Suppress the graphics */
ods exclude Statistics /* Not to generate output for the overall population */
TITLE "PERSON-LEVEL ESTIMATES ON EXPENDITURES AND USE FOR NARCOTIC ANALGESICS or NARCOTIC COMBOS, 2017";
PROC SURVEYMEANS DATA=WORK.FY NOBS SUMWGT SUM MEAN STDERR SUM;
  VAR N_PHRCHASE TOT OOP THIRD_PAYER ;
  STRATA  VARSTR ;
  CLUSTER VARPSU;
  WEIGHT  PERWT17F;
  DOMAIN  SUBPOP("PERSONS WITH 1+ Narcotic etc");
  FORMAT SUBPOP SUBPOP.;
 RUN;